In [1]:
import polars as pl

In [3]:
df = pl.read_parquet("../data/predictions/2024-04-22 11:40:brandon-terrace:IRVE.parquet")

In [37]:
df

predictions,targets,sequence_id,event_id
list[f32],i64,str,str
"[0.000043, 0.999957]",1,"""Jessica Leon""","""2018-04-06 14:…"
"[0.000003, 0.999997]",1,"""Jessica Leon""","""2017-01-04 18:…"
"[0.000007, 0.999993]",1,"""Jessica Leon""","""2018-10-21 11:…"
"[0.000018, 0.999982]",1,"""Jessica Leon""","""2017-03-05 16:…"
"[0.000005, 0.999995]",1,"""Jessica Leon""","""2018-10-24 18:…"
…,…,…,…
"[0.000009, 0.999991]",1,"""Zachary Nelson…","""2016-11-02 11:…"
"[0.000005, 0.999995]",1,"""Judith Frank""","""2019-10-11 20:…"
"[0.000007, 0.999993]",1,"""Judy Orozco""","""2020-01-18 17:…"


In [63]:

from zlib import crc32

seed = (
    pl.col("sequence_id")
    .cast(pl.String)
    .map_elements(lambda x: float(crc32(str.encode(f"{x}_")) & 0xFFFFFFFF), return_dtype=pl.Float32)
    .mul(1 / 2**32)
)

filtered = df.filter((seed < 0.1) | (pl.col("targets") == 0))

In [64]:
df.select(seed).describe()

statistic,sequence_id
str,f64
"""count""",5.232384e6
"""null_count""",0.0
"""mean""",0.516446
"""std""",0.286869
"""min""",0.000161
"""25%""",0.286851
"""50%""",0.535694
"""75%""",0.765502
"""max""",0.992863


In [90]:
yhat = filtered.get_column('predictions').to_list()
y = filtered.select(targets=pl.col("targets").cast(pl.Boolean).not_().cast(pl.Int32)).get_column('targets').to_numpy()

In [91]:
from sklearn.metrics import average_precision_score as ap

In [92]:
ap(y_true=y, y_score=yhat)


ValueError: y should be a 1d array, got an array of shape (569167, 2) instead.

In [89]:
y

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)